<a href="https://colab.research.google.com/github/GarlandZhang/hairy_gan/blob/master/style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.preprocessing import image as kp_image

from keras.applications.vgg19 import VGG19
from keras.models import Model, Sequential
from keras.layers import Dense, Input
from keras.optimizers import Adam
from keras import backend as K

from PIL import Image

import numpy as np

import os
tf.compat.v1.disable_v2_behavior()

In [ ]:
# preprocess image
def load_img(img_path):
  img = Image.open(img_path)

  # resize to max dimension
  max_dim = 512
  img_size = max(img.size)
  scale = max_dim / img_size 
  img = img.resize((round(img.size[0] * scale), round(img.size[1] * scale)), Image.ANTIALIAS)

  img = np.expand_dims(img, axis=0)

  # required step to run vgg19
  out = tf.keras.applications.vgg19.preprocess_input(img)

  return out

# postprocess image
def postprocess_img(processed_img):
  img = processed_img.copy()

  img[:, :, 0] += 103.939
  img[:, :, 1] += 116.779
  img[:, :, 2] += 123.68

  img = img[:, :, ::-1]

  img = np.clip(img, 0, 255).astype('uint8')

  return img

In [ ]:
# build loss functions from scratch

def gram_matrix(input_tensor):
  # 3D => 2D matrix: nh * nw * nc => nc * (nh * nw)
  channels = int(input_tensor.shape[-1])

  a = tf.reshape(input_tensor, [-1, channels])
  n = tf.shape(a)[0]

  gram = tf.matmul(a, a, transpose_a=True)

  return gram

def get_style_loss(gram_target, base_style): # base_style is generated layer output, gram_target is style layer output
  _, height, width, channels = base_style.get_shape().as_list()

  print(f'gram_target: {gram_target.get_shape()} | base_style: {base_style.get_shape()}')

  gram_style = gram_matrix(base_style)

  return tf.reduce_mean(tf.square(gram_style - gram_target)) / (channels**2) #(4.0 * (channels ** 2) * (width * height) ** 2)

def get_content_loss(target, content): # content is new generated image, target is original image
  return tf.reduce_mean(tf.square(content - target)) / 2

In [ ]:
# get feature outputs for style and content images
def get_feature_representations(model, content_img_path, style_img_path):

  content_img = load_img(content_img_path)
  style_img = load_img(style_img_path)

  content_outputs = model.predict(content_img)[num_style_layers:] # only get content features of content image (dont care about style)
  style_outputs = model.predict(style_img)[:num_style_layers] # only get style features of style image

  content_features = [ np.array([content_layer[0]]) for content_layer in content_outputs ]
  style_features = [ np.array([style_layer[0]]) for style_layer in style_outputs ]

  return style_features, content_features

In [ ]:
def build_model(loss_weights):
  vgg19 = VGG19(weights=None, include_top=False)
  vgg19.trainable = False # pretrained; don't touch

  content_model_outputs = [vgg19.get_layer(layer).output for layer in content_layers]
  style_model_outputs = [vgg19.get_layer(layer).output for layer in style_layers]

  model_outputs = style_model_outputs + content_model_outputs # must combine output...we cannot output lists as outputs, we must have a list of outputs if anything

  model = Model(inputs=vgg19.input, outputs=model_outputs)

  style_weight, content_weight = loss_weights

  losses = [get_style_loss for style_layer in style_layers] + [get_content_loss for content_layer in content_layers]

  # calculate loss_weights
  weight_per_style_layer = 1.0 / float(num_style_layers) * style_weight
  weight_per_content_layer = 1.0 / float(num_content_layers) * content_weight

  loss_weights = [weight_per_style_layer for style_layer in style_layers] + [weight_per_content_layer for content_layer in content_layers]

  model.compile(loss=losses, loss_weights=loss_weights, optimizer=Adam())

  vgg19.load_weights(vgg_weights_path)

  return model, vgg19 # takes part of model up to the last output 

In [ ]:
def run_style_transfer(num_iterations=200, content_weight=0.1, style_weight=0.9):
  # sess = tf.compat.v1.Session()
  # tf.compat.v1.keras.backend.set_session(sess)

  loss_weights = (style_weight, content_weight)

  model, vgg19 = build_model(loss_weights)

  # the loss is only updated if the current loss is better (i.e lesser) than the previous loss
  best_loss, best_img = None, None

  # get layer outputs from the content image and style image
  style_features, content_features = get_feature_representations(model, content_img_path, style_img_path)

  # get gram matrix values to prepare for total loss calc
  # gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]
  gram_style_features = [tf.expand_dims(gram_matrix(style_feature), 0) for style_feature in style_features]

  print(f'content_feature: {content_features[0].shape}')
  print(f'gram_style_feature[1]: {gram_style_features[1].shape}')

  # content_features = [np.expand_dims(feature, axis=0) for feature in content_features]

  # get layer outputs for generated image
  generated_image = load_img(content_img_path)

  for i in range(num_iterations):
    history = model.fit(generated_image, gram_style_features + content_features, steps_per_epoch=10)
    loss = history.history['loss']

    if best_loss == None or loss < best_loss:
      best_loss = loss
      best_img = postprocess_img(generated_img)

      print(f'best loss: {best_loss}')

    if (i + 1) % 100 == 0:
      output = Image.fromarray(best_img)
      output.save(os.path.join(project_path, f'{i + 1}-{save_img_path}'))

      # save model
      model.save_weights(os.path.join(project_path, 'style_transfer.weights'))

  # how to use this?
  # VGG default normalization
  norm_means = np.array([103.939, 116.779, 123.68])
  min_vals = -norm_means
  max_vals = 255 - norm_means

  return best_img, best_loss

In [ ]:
# list of layers to caulcate for content and style loss
content_layers = ['block3_conv3']
style_layers = ['block1_conv1', 'block2_conv2', 'block4_conv3']

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

project_path = '/content/drive/My Drive/hairy_gan'
content_img_path = os.path.join(project_path, 'content.jpg')
style_img_path = os.path.join(project_path, 'style.jpg')
save_img_path = os.path.join(project_path, 'generated.jpg')

vgg_weights_path = os.path.join(project_path, 'vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [ ]:
best, best_loss = run_style_transfer()

gram_target: (?, ?, ?, ?) | base_style: (?, ?, ?, 64)
gram_target: (?, ?, ?, ?) | base_style: (?, ?, ?, 128)
gram_target: (?, ?, ?, ?) | base_style: (?, ?, ?, 512)
content_feature: (1, 85, 128, 256)
gram_style_feature[1]: (1, 128, 128)
Epoch 1/1


InvalidArgumentError: ignored

In [ ]:
constant_float_ext = tf.constant([[[1.,1.,1.],[2.,2.,2.],[3.,3.,3.]],[[4.,4.,4.],[5.,5.,5.],[6.,6.,6.]]])
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())
print(sess.run(tf.compat.v1.reduce_mean(constant_float_ext, axis=[0, 1, 2])))

3.5


In [ ]:
input = Input(shape=(128, 128, 3))
layer1 = Dense(3)(input)
layer2 = Dense(3)(layer1)
layer3 = Dense(3)(layer2)
mod = Model(input=input, output=layer3)
mod.summary()

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 128, 128, 3)       0         
_________________________________________________________________
dense_16 (Dense)             (None, 128, 128, 3)       12        
_________________________________________________________________
dense_17 (Dense)             (None, 128, 128, 3)       12        
_________________________________________________________________
dense_18 (Dense)             (None, 128, 128, 3)       12        
Total params: 36
Trainable params: 36
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """


In [ ]:
mod2 = Model(input=input, output=[layer1, layer2])
mod2.compile(loss=mod_loss, optimizer=Adam())
mod2.fit(img, [img, img])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


Epoch 1/1
1/1 [==============================] - 0s 405ms/step - loss: 149845.9062 - dense_16_loss: 83554.4141 - dense_17_loss: 66291.5000


In [ ]:
def mod_loss(y_true, y_pred):
  return tf.reduce_mean(tf.square(y_true - y_pred))

def rand_loss(layer_out):
  def loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(layer_out))
  return loss

In [ ]:
import cv2
img = cv2.imread(content_img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (128, 128))
img = np.expand_dims(img, axis=0)

In [ ]:
vgg19 = VGG19(weights=None, include_top=False)
vgg19.trainable = False # pretrained; don't touch

style_model_outputs = [vgg19.get_layer(layer).output for layer in style_layers]
content_model_outputs = [vgg19.get_layer(layer).output for layer in content_layers]

model_outputs = style_model_outputs + content_model_outputs # must combine output...we cannot output lists as outputs, we must have a list of outputs if anything

model = Model(inputs=vgg19.input, outputs=model_outputs)

In [ ]:
img2 = load_img(content_img_path)
outputs = model.predict(img2)
content_outputs = outputs[num_style_layers:]
style_outputs = outputs[:num_style_layers]
content_features = [ np.array([content_layer[0]]) for content_layer in content_outputs]
style_features = [ np.array([style_layer[0]]) for style_layer in style_outputs ]

gram_style_features = [tf.expand_dims(gram_matrix(style_feature), 0) for style_feature in style_features]
model_features = gram_style_features + content_features

In [ ]:
def vgg_loss(y_true, y_pred):
  gram_style = gram_matrix(y_pred[0])

  return tf.argmax(tf.square(gram_style - y_true)) / 2

def vgg_loss2(gram_target, base_style): # base_style is generated layer output, gram_target is style layer output
  _, height, width, channels = base_style.get_shape().as_list()

  gram_style = gram_matrix(base_style)

  return tf.reduce_mean(tf.square(gram_style - gram_target)) / (channels**2) #(4.0 * (channels ** 2) * (width * height) ** 2)


In [ ]:
model.compile(loss=[vgg_loss] * len(model_outputs), optimizer=Adam())

In [ ]:
model.fit(img2, model_features, steps_per_epoch=1)

ValueError: ignored

In [ ]:
model_features[-1].shape

(1, 85, 128, 256)

In [ ]:
np.array(1).tolist()

1